# Transformer based models

### Importing the required libraries

In [1]:
import io
import numpy as np
# from google_trans_new import google_translator 
from scipy.spatial.distance import cosine
import json 
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk import word_tokenize
from itertools import groupby 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install -U torch
import torch
!pip install -U transformers
!pip install -U sentence-transformers
import transformers
from sentence_transformers import SentenceTransformer,util
import numpy as np

In [5]:
# model = SentenceTransformer('stsb-xlm-r-multilingual')
# model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
# model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')

In [6]:
sentence1 = "I robbed bank"
sentence2 = "मुझे कुत्ते अच्छे लगते हैं"
# encode sentences to get their embeddings
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)
# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
print("Sentence 1:", sentence1)
print("Sentence 2:", sentence2)
print("Similarity score:", cosine_scores.item())

Sentence 1: I robbed bank
Sentence 2: मुझे कुत्ते अच्छे लगते हैं
Similarity score: 0.1202767863869667


### Loading data from json file

In [14]:
import json 
# Opening JSON file 
f = open('/content/drive/MyDrive/test_data_v2.json',encoding = 'utf-8') 
  
# returns JSON object as  
# a dictionary 
data = json.load(f) 
d = {}
d = data
  
# Closing file 
f.close()

{'Q12053145': {'entity_id': 'Q12053145',
  'sentence': 'दीप्ती भटनागर हिन्दी फ़िल्मों की एक अभिनेत्री हैं',
  'triples': [{'object': 'female',
    'predicate': 'sex or gender',
    'subject': 'Deepti Bhatnagar'},
   {'object': 'actor',
    'predicate': 'occupation',
    'subject': 'Deepti Bhatnagar'}]},
 'Q12333184': {'entity_id': 'Q12333184',
  'sentence': 'न् १९७५ जुलाई ५) एक नेपाली अभिनेत्री,मॉडल और फिल्म निर्माता हैं',
  'triples': [{'object': 'female',
    'predicate': 'sex or gender',
    'subject': 'Rekha Thapa'},
   {'object': 'Nepal',
    'predicate': 'country of citizenship',
    'subject': 'Rekha Thapa'},
   {'object': 'actor', 'predicate': 'occupation', 'subject': 'Rekha Thapa'},
   {'object': 'film actor',
    'predicate': 'occupation',
    'subject': 'Rekha Thapa'}]},
 'Q12443074': {'entity_id': 'Q12443074',
  'sentence': 'ब्रह्म भारद्वाज हिन्दी फ़िल्मों के एक अभिनेता हैं',
  'triples': [{'object': 'male',
    'predicate': 'sex or gender',
    'subject': 'Brahm Bhardwaj'}

In [ ]:
### Getting a_c_p for getting all the triples
# Opening JSON file 
f = open('/content/drive/MyDrive/a_c_p.json',) 
data = json.load(f) 
acp = {}
acp = data
f.close() 

### Storing sentences and triples from one article
Here we store all the sentences and the triples from one article. The article is about the actress 'Kalpana'. We display 5 sentences and 5 triples corresponding to her here.

### Obtaining sentence and triple embeddings
We obtain the sentence embeddings by taking each sentence, obtaining word embeddings for each word in the sentence, and averaging the word embeddings. For triple embeddings , we average the embeddings for each word in the triple.

In [ ]:
# Matching Triples with sentences
actors_test, cricketers_test, politicians_test = d['actors'], d['cricketers'], d['politicians']
actors, cricketers, politicians = actors_test, cricketers_test, politicians_test

# Creating the dictionary for the test data where key = sentence and value = list of matching triples
actors_test_dict, cricketers_test_dict, politicians_test_dict = {}, {}, {}
for l in actors_test:
    for k, v in l.items():
        if k == 'sentence':
            sentence = v
        if k == 'triples':
            triple_list = v
    t = [(e['subject'], e['predicate'], e['object']) for e in triple_list]
    actors_test_dict[sentence] = set(t)

for l in cricketers_test:
    for k, v in l.items():
        if k == 'sentence':
            sentence = v
        if k == 'triples':
            triple_list = v
    t = [(e['subject'], e['predicate'], e['object']) for e in triple_list]
    cricketers_test_dict[sentence] = set(t)

for l in politicians_test:
    for k, v in l.items():
        if k == 'sentence':
            sentence = v
        if k == 'triples':
            triple_list = v
    t = [(e['subject'], e['predicate'], e['object']) for e in triple_list]
    politicians_test_dict[sentence] = set(t)
    
len(actors_test_dict),len(cricketers_test_dict),len(politicians_test_dict)

(60, 13, 30)

In [ ]:
# We had got a_c_p.json. We keep only the relevant triples by filtering by entity id in test annotated data
actors_trip, cricketers_trip, politician_trip = [], [], []
actors_sent, cricketers_sent, politician_sent = [], [], []

# Putting actors,cricketers and politicians from a_c_p
act, cric, pol = acp['a'], acp['c'], acp['p']

############### Actors #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in actors_test:
    eid = e['entity_id']
    for ele in act:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = act[ele]['triples']
            subject = act[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            actors_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id

for eid in entity_tracking:
    sentence_list = []
    for e in actors_test:
        if eid == e['entity_id']:
            sentence = e['sentence']
            sentence_list.append(sentence)
    actors_sent.append(sentence_list)


############### Cricketers #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in cricketers_test:
    eid = e['entity_id']
    for ele in cric:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = cric[ele]['triples']
            subject = cric[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            cricketers_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id

for eid in entity_tracking:
    sentence_list = []
    for e in cricketers_test:
        if eid == e['entity_id']:
            sentence = e['sentence']
            sentence_list.append(sentence)
    cricketers_sent.append(sentence_list)


############### Politicians #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in politicians_test:
    eid = e['entity_id']
    for ele in pol:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = pol[ele]['triples']
            subject = pol[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            politician_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id

for eid in entity_tracking:
    sentence_list = []
    for e in politicians_test:
        if eid == e['entity_id']:
            sentence = e['sentence']
            sentence_list.append(sentence)
    politician_sent.append(sentence_list)

In [ ]:
len(actors_sent), len(cricketers_sent),len(politician_sent)

(34, 13, 27)

In [ ]:
len(actors_trip), len(cricketers_trip),len(politician_trip)

(34, 13, 27)

In [ ]:
actors_trip[0]

[('Karan Tacker',
  'image',
  'http://commons.wikimedia.org/wiki/Special:FilePath/Karan%20Tacker%20at%20the%2011th%20Star%20Parivaar%20Awards%202013.jpg'),
 ('Karan Tacker', 'place of birth', 'Mumbai'),
 ('Karan Tacker', 'sex or gender', 'male'),
 ('Karan Tacker', 'country of citizenship', 'India'),
 ('Karan Tacker', 'instance of', 'human'),
 ('Karan Tacker', 'occupation', 'actor'),
 ('Karan Tacker', 'occupation', 'television actor'),
 ('Karan Tacker', 'occupation', 'film actor'),
 ('Karan Tacker', 'Commons category', 'Karan Tacker'),
 ('Karan Tacker', 'date of birth', '1986-05-11T00:00:00Z'),
 ('Karan Tacker', 'given name', 'Karan'),
 ('Karan Tacker', 'Twitter username', 'karantacker'),
 ('Karan Tacker', 'Instagram username', 'karantacker'),
 ('Karan Tacker', 'Facebook ID', 'tackerkaran'),
 ('Karan Tacker', 'social media followers', '1067819')]

In [ ]:
def matches(sentences,triples):       # Input is a sentence list and triple list for one article
    matches_dict = {}                 # Dictionary to store the sentences with the matching triples. key = sent, value = triple
    for sent in sentences:            # For each sent in sentences
        ent_matchlist = []
        sent_embed = model.encode(sent, convert_to_tensor=True)
        for ent in triples:
            pred = ent[1]
            obj = ent[2]
            # print(pred)
            ent_embed = model.encode(pred + " " + obj, convert_to_tensor=True)
            cosine_score = util.pytorch_cos_sim(sent_embed, ent_embed)
            similarity = cosine_score.item()
            
            if similarity <= 0.35:
              # For evaluation of precision and recall, keep the below 3 lines commented out. They are to append score to matching triples
                ent = list(ent)
                ent.append(similarity)
                ent = tuple(ent)
                ent_matchlist.append(ent)
        if len(ent_matchlist)>0:    
            matches_dict[sent] = set(ent_matchlist)
    return matches_dict

In [ ]:
matches_act = {}
for sent_list, triple_list in tqdm(zip(actors_sent, actors_trip)):
    for k, v in matches(sent_list, triple_list).items():
        matches_act[k] = v

matches_cric = {}
for sent_list, triple_list in tqdm(zip(cricketers_sent, cricketers_trip)):
    for k, v in matches(sent_list, triple_list).items():
        matches_cric[k] = v

matches_pol = {}
for sent_list, triple_list in tqdm(zip(politician_sent, politician_trip)):
    for k, v in matches(sent_list, triple_list).items():
        matches_pol[k] = v

34it [02:30,  4.43s/it]
13it [00:20,  1.61s/it]
27it [01:13,  2.73s/it]


### Evaluation : Precision and Recall

In [ ]:
def evaluate(test_dict, matches_dict):
    sum_prec = 0
    for key, val in matches_dict.items():
        tp, fp = 0, 0
        for k, v in test_dict.items():
            # If sentence matches
            if k == key:
                for ent in v:
                    for trip in val:
                        if ent == trip:
                            tp = tp + 1
                for trip in val:
                    flag = 0
                    for ent in v:
                        if ent == trip:
                            flag = 1
                            break
                    if flag == 0:
                        fp = fp + 1
                break
        if (tp+fp) != 0:
            prec = tp/(tp + fp)
        else:
            prec = 0
        sum_prec = prec + sum_prec

    sum_rec = 0
    for k, v in test_dict.items():
        rec = 0
        tp, fp = 0, 0
        for key, val in matches_dict.items():
            # If sentence matches
            if k == key:
                for ent in v:
                    for trip in val:
                        if ent == trip:
                            tp = tp + 1
                for trip in val:
                    flag = 0
                    for ent in v:
                        if ent == trip:
                            flag = 1
                            break
                    if flag == 0:
                        fp = fp + 1
                break
        rec = tp/len(v)
        sum_rec = rec + sum_rec

    avg_rec, avg_prec = sum_rec/len(test_dict), sum_prec/len(matches_dict)
    return avg_rec, avg_prec


avg_rec_act, avg_prec_act = evaluate(actors_test_dict, matches_act)
avg_rec_cric, avg_prec_cric = evaluate(cricketers_test_dict, matches_cric)
avg_rec_pol, avg_prec_pol = evaluate(politicians_test_dict, matches_pol)

In [ ]:
(avg_rec_act,avg_prec_act), (avg_rec_cric, avg_prec_cric) , (avg_rec_pol, avg_prec_pol)

((0.0, 0.0), (0.0, 0.0), (0.0, 0.0))

In [ ]:
AverageRecall = (avg_rec_act + avg_rec_cric + avg_rec_pol)/3
AveragePrecision = (avg_prec_act + avg_prec_cric + avg_prec_pol)/3

AverageRecall, AveragePrecision

(0.581994301994302, 0.4492796495956874)

### Output

In [ ]:
len(matches_act),len(matches_cric),len(matches_pol)

(60, 13, 29)

So, 7 sentences didn't find any matches at all.Let's see which ones

In [ ]:
for sent in politicians_test_dict:
  if sent not in matches_pol:
    print(sent," : ", politicians_test_dict[sent] )

करन टेकर एक भारतीय अभिनेता हैं  :  {('Karan Tacker', 'occupation', 'actor'), ('Karan Tacker', 'country of citizenship', 'India')}
मार्च 1976 में वे राज्यसभा के लिए चुनी गईं  :  {('Ambika Soni', 'position held', 'Member of Rajya Sabha')}
16 जून 2012 को, लियू अंतरिक्ष में जाने वाली पहली चीन महिला बनी  :  {('Liu Yang', 'sex or gender', 'female'), ('Liu Yang', 'occupation', 'astronaut')}
वह आंतरिक मामलों और संचार मंत्री भी रह चुकी हैं  :  {('Prakash Karat', 'position held', 'Minister of Internal Affairs and Communications')}
उन्हें 1 अक्टूबर 1990 से 1 दिसंबर 1993 के बीच, तुवालू की रानी, एलिज़ाबेथ द्वितीय द्वारा, तुवालू के गवर्नर-जनरल यानि महाराज्यपाल के पद पर नियुक्त किया गया था  :  {('Toaripi Lauti', 'position held', 'Prime Minister of Tuvalu')}


Saving as json

In [ ]:
for sent in actors_test_dict:
  actors_test_dict[sent] = list(actors_test_dict[sent])
for sent in cricketers_test_dict:
  cricketers_test_dict[sent] = list(cricketers_test_dict[sent])
for sent in politicians_test_dict:
  politicians_test_dict[sent] = list(politicians_test_dict[sent])

for sent in matches_act:
  matches_act[sent] = list(matches_act[sent])
for sent in matches_cric:
  matches_cric[sent] = list(matches_cric[sent])
for sent in matches_pol:
  matches_pol[sent] = list(matches_pol[sent])

In [ ]:
# saving the input test files
# with open("/content//drive//MyDrive//Transformer missed outputs//actors_test.json", "w") as outfile: 
#     json.dump(actors_test_dict,outfile)
# with open("/content/drive/MyDrive/Transformer missed outputs/cricketers_test.json", "w") as outfile: 
#     json.dump(cricketers_test_dict, outfile)
# with open("/content/drive/MyDrive/Transformer missed outputs/politicians_test.json", "w") as outfile: 
#     json.dump(politicians_test_dict, outfile)

#saving the output files
with open("/content/drive/MyDrive/Transformer missed outputs/actors_matches.json", "w") as outfile: 
    json.dump(matches_act, outfile)
with open("/content/drive/MyDrive/Transformer missed outputs/cricketers_matches.json", "w") as outfile: 
    json.dump(matches_cric, outfile)
with open("/content/drive/MyDrive/Transformer missed outputs/politicians_matches.json", "w") as outfile: 
    json.dump(matches_pol, outfile)

#### We do find quite a few of the matching triples to be relevant to the sentence. But, there are a few irrelevant matches as well.
Upon analysis, we think the word overlap is working better than the vector similarity approach. A possible reason can be that when we simply average out the words in a sentence, and when we average out the words in the triples and then find the similarity between these two averages, some semantic information is lost. So, triples that should have been irrelevant are also found as similar. As the word overlap method is a strictly string overlap, the relevance is much stronger.